In [ ]:
%pwd

In [ ]:
%pip install pandas

In [ ]:
import pandas as pd
sample = pd.read_csv('/home/kwy00/ysy/sample_submission.csv')
sample

In [ ]:
import os

test_filepath = '/home/kwy00/ysy/test/'
train_filepath = '/home/kwy00/ysy/train/'
# valid_filepath = '/home/kwy00/ysy/valid/' # after running preprocessing

test_list = os.listdir(test_filepath)
train_list = os.listdir(train_filepath)
# valid_list = os.listdir(valid_filepath)

In [ ]:
test_file_count = len(test_list)
train_file_count = len(train_list)
# valid_file_count = len(valid_list)

print("number of test file : ", test_file_count)
print("number of train file : ", train_file_count)
# print("number of valid file : ", valid_file_count")

print("number of images in train : ", (int)(train_file_count/2)) # rest half consist of txt file which contains label info.
# print("number of images in valid : ", (int)(valid_file_count/2))

print("total image file : ", test_file_count+(int)(train_file_count/2))
# print("total image file : ", test_file_count+(int)(train_file_count/2)+(int)(valid_file_count/2))

In [ ]:
with open("/home/kwy00/ysy/classes.txt") as f:
  class_label = f.read()
print(class_label)

# chevrolet : malibu      2012~2016   000
#             malibu      2017~2019   001
#             spark       2016~2021   002
#             trailblazer 2021        003
#             trax        2017~2019   004
# genesis     g80         2016~2020   005
#             g80         2021        006
#             gv80        2020        007
# hyundai     avante      2011~2015   008
#             avante      2020        009
#             grandeur    2011~2016   010
#             grandstarex 2018~2020   011
#             ioniq       2016~2019   012
#             sonata      2004~2009   013
#             sonata      2010~2014   014
#             sonata      2019~2020   015
# kia         carnival    2015~2020   016
#             carnival    2021        017
#             k5          2010~2015   018
#             k5          2020        018
#             k7          2016~2020   020
#             mohave      2020        021
#             morning     2004~2010   022
#             morning     2011~2016   023
#             ray         2012~2017   024
#             sorrento    2015~2019   025
#             sorrento    2020        026
#             soul        2014~2018   027
#             sportage    2016~2020   028
#             stonic      2017~2019   029
# renault     sm3         2015~2018   030
#             xm3         2020        031
# ssangyong   korando     2019~2020   032
#             tivoli      2016~2020   033

In [ ]:
from PIL import Image
ex_image = Image.open(train_filepath + '/syn_00010.png')
ex_image

In [ ]:
png_file_list = [] # for train
txt_file_list = []
for files in train_list:
  if (files[-3:] == 'txt'):
    txt_file_list.append(files)
  else:
    png_file_list.append(files)

print(len(png_file_list))
print(png_file_list)

In [ ]:
%pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
start_index = 50
print_count = 10

columns = 3
rows = (int)(print_count/columns) + 1
plt.figure(figsize=(20, 20))

image_index = 0
for i in range(start_index, start_index+print_count):
  image_index = image_index + 1
  title = "Image{}".format(image_index)
  image = Image.open(train_filepath + png_file_list[i]) # bring image from folder
  plt.subplot(rows, columns, image_index) # subplot
  plt.title(title)

  plt.xticks([])
  plt.yticks([])
  plt.imshow(image)
plt.show()

In [ ]:
# check image features
from PIL import Image

image = Image.open('./train/syn_00001.png')
print("image_2 format: ", image.format)
print("image_2 mode: ", image.mode)
print("image_2 size: ", image.size)

image2 = Image.open('./train/syn_00002.png')
print("image_2 format: ", image2.format)
print("image_2 mode : ", image2.mode)
print("image_2 size : ", image2.size)

# all images have same size for 1920x1040 and RGBA mode, png format
for png_img in png_file_list:
    img = Image.open('./train/' + png_img)
    if img.size != (1920, 1040):
        print("image size different! ", png_img)
    if img.mode != 'RGBA':
        print("image mode different! ", png_img)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

# for data augmentation and image drawing and reading
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
def draw_boxes_on_image(image_path, annotation_path):
    # 이미지 불러오기
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # txt 파일에서 Class ID와 Bounding Box 정보 읽기
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        values = list(map(float, line.strip().split(' ')))
        class_id = int(values[0])
        x_min, y_min = int(round(values[1])), int(round(values[2]))
        x_max, y_max = int(round(max(values[3], values[5], values[7]))), int(round(max(values[4], values[6], values[8])))

        # 이미지에 바운딩 박스 그리기
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        cv2.putText(image, str(class_id), (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 이미지와 바운딩 박스 출력
    plt.figure(figsize=(25, 25))
    plt.imshow(image)
    plt.show()

# 파일 경로 설정
image_file = './train/syn_00001.png'
annotation_file = './train/syn_00001.txt'

# 함수 실행
draw_boxes_on_image(image_file, annotation_file)

In [ ]:
###################################################
#               define augmentation               #
###################################################

# reference : https://albumentations.ai/docs/api_reference/augmentations/transforms/
    
randomly_shuffle_color_transforms = A.Compose([
        A.augmentations.transforms.ChannelShuffle(0.25), # 25% randomly shuffles
        A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

contrast_enhancement_transforms = A.Compose([
        A.augmentations.transforms.CLAHE(clip_limit=3.0, p=0.9), # clip_limit=3.0, p=0.5 is default
        A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

image_embossing_transforms = A.Compose([
        A.augmentations.transforms.Emboss(alpha=(0.2, 0.3), p=0.8),
        A.resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

default_transforms = A.Compose([
        # A.augmentations.transforms.ISONoise(p=1.0),
        A.Resize(CFG['IMG_HEIGHT_SIZE'], CFG['IMG_WIDTH_SIZE']), # baseline resize to (512 x 512)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    

default_transforms_v2 = A.Compose([
        A.augmentations.geometric.transforms.HorizontalFlip(p=0.2),
        A.Resize(CFG['IMG_HEIGHT_SIZE'], CFG['IMG_WIDTH_SIZE']), # baseline resize to (512 x 512)    
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

decreasing_image_quality_transforms = A.Compose([
        # A.augmentations.transforms.Downscale(scale_min=0.5, scale_max=0.5, p=0.75),
        A.augmentations.transforms.PixelDropout(dropout_prob=0.5, drop_value=0, p=0.5),
        # A.augmentations.transforms.GaussNoise(p=0.2),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

colorjitter_transforms = A.Compose([
        A.augmentations.transforms.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.2),
                                        saturation=(0.5, 1.2), hue=0.0,
                                        always_apply=False, p=1.0),
        A.Resize(CFG['IMG_HEIGHT_SIZE'], CFG['IMG_WIDTH_SIZE'])
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

bboxsafe_cropping_transforms = A.Compose([
            A.augmentations.crops.transforms.BBoxSafeRandomCrop(p=1.0),
            A.Resize(CFG['IMG_HEIGHT_SIZE'], CFG['IMG_WIDTH_SIZE']),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'],
                                    min_area=2048, min_visibility=0.2))

bbox_nearcrop_transforms = A.Compose([
            A.augmentations.crops.transforms.RandomCropNearBBox(max_part_shift=(0.3, 0.3), p=1.0),
            A.Resize(CFG['IMG_HEIGHT_SIZE'], CFG['IMG_WIDTH_SIZE']),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'],
                                    min_area=2048, min_visibility=0.2))


In [ ]:
ROOT = './train'

IMGS = sorted(glob.glob(ROOT+'/*.png'))
BOXES = sorted(glob.glob(ROOT+'/*.txt'))

In [ ]:
def parse_boxes(self, box_path):
    with open(box_path, 'r') as file:
        lines = file.readlines()

    boxes = []
    labels = []

    for line in lines:
        values = list(map(float, line.strip().split(' ')))
        class_id = int(values[0])
        x_min, y_min = int(round(values[1])), int(round(values[2]))
        x_max, y_max = int(round(max(values[3], values[5], values[7]))), int(round(max(values[4], values[6], values[8])))

        boxes.append([x_min, y_min, x_max, y_max])
        labels.append(class_id)

    return torch.tensor(boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

In [ ]:
def drawing_tool(image, boxes, labels):
    
    for box, label in (boxes,labels):    
        
        class_id = label
        x_min, y_min = int(round(box[1])), int(round(box[2]))
        x_max, y_max = int(round(max(box[3], box[5], box[7]))), int(round(max(box[4], box[6], box[8])))

        # 이미지에 바운딩 박스 그리기
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        cv2.putText(image, str(class_id), (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 이미지와 바운딩 박스 출력
    plt.figure(figsize=(25, 25))
    plt.imshow(image)
    plt.show()

In [ ]:
IDX = 1
    
img_path = IMGS[IDX]
file_name = img_path.split('/')[-1]

img = cv2.imread(IMGS[IDX])

# convert image color mode (BGR -> RGB)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.uint8)

height, width = img.shape[0], img.shape[1]
box_path = BOXES[IDX]
boxes, labels = parse_boxes(box_path)

labels += 1 # Background = 0, add background label as 0

transformed = colorjitter_transforms(image=img, bboxes=boxes, labels=labels)
img, boxes, labels = transformed["image"], transformed["bboxes"], transformed["labels"]

In [ ]:
drawing_tool(img, boxes, labels)